In [ ]:
# Install dependencies (run once)
!pip install tensorflow transformers tf2onnx onnx onnxruntime numpy -q

In [ ]:
import os
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer

print(f"TensorFlow: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## Configuration

Adjust these parameters based on your hardware and dataset size.

In [ ]:
# Model Configuration
MAX_SEQ_LENGTH = 40
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_LAYERS = 2

# Training Configuration  
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 0.001

# Output paths
MODEL_PATH = "cerebros_model.keras"
ONNX_PATH = "cerebros_model.onnx"

print(f"Config: seq_len={MAX_SEQ_LENGTH}, embed_dim={EMBEDDING_DIM}, hidden={HIDDEN_DIM}")

## Load Tokenizer

Using HuggingFace SmolLM tokenizer (can substitute any tokenizer).

In [ ]:
# Load tokenizer from HuggingFace
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
VOCAB_SIZE = tokenizer.vocab_size

# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print(f"Tokenizer loaded: vocab_size={VOCAB_SIZE}")
print(f"Pad token: {tokenizer.pad_token} (id: {tokenizer.pad_token_id})")

## Interleaved RoPE (iRoPE) Layer

Custom positional encoding used by Cerebros architecture.

In [ ]:
@tf.keras.utils.register_keras_serializable(package='cerebros', name='InterleavedRoPE')
class InterleavedRoPE(tf.keras.layers.Layer):
    """
    Interleaved Rotary Position Embedding (iRoPE)
    Applies rotary embeddings with interleaved real/imaginary components.
    """
    
    def __init__(self, dim, max_seq_len=512, base=10000.0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.max_seq_len = max_seq_len
        self.base = base
        
    def build(self, input_shape):
        # Precompute frequencies
        half_dim = self.dim // 2
        freqs = 1.0 / (self.base ** (tf.range(0, half_dim, dtype=tf.float32) / half_dim))
        positions = tf.range(self.max_seq_len, dtype=tf.float32)
        angles = tf.einsum('i,j->ij', positions, freqs)
        
        self.cos_cached = tf.cos(angles)
        self.sin_cached = tf.sin(angles)
        super().build(input_shape)
        
    def call(self, x):
        seq_len = tf.shape(x)[1]
        cos = self.cos_cached[:seq_len]
        sin = self.sin_cached[:seq_len]
        
        # Split into pairs for rotation
        x1 = x[..., 0::2]
        x2 = x[..., 1::2]
        
        # Apply rotation
        rotated_x1 = x1 * cos - x2 * sin
        rotated_x2 = x1 * sin + x2 * cos
        
        # Interleave back
        result = tf.stack([rotated_x1, rotated_x2], axis=-1)
        result = tf.reshape(result, tf.shape(x))
        
        return result
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'dim': self.dim,
            'max_seq_len': self.max_seq_len,
            'base': self.base
        })
        return config

print("✓ InterleavedRoPE layer defined")

## Build the Model

A simple transformer-style architecture with:
- Token embeddings
- iRoPE positional encoding
- Dense layers
- Vocabulary prediction head

In [ ]:
def build_cerebros_model(vocab_size, max_seq_len, embed_dim, hidden_dim, num_layers):
    """Build a Cerebros-style LLM model."""
    
    # Input layer
    inputs = tf.keras.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_ids")
    
    # Token embedding
    x = tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embed_dim,
        name="token_embedding"
    )(inputs)
    
    # Apply iRoPE positional encoding
    x = InterleavedRoPE(dim=embed_dim, max_seq_len=max_seq_len, name="irope")(x)
    
    # Transformer-style processing
    for i in range(num_layers):
        # Self-attention approximation via dense layers
        residual = x
        x = tf.keras.layers.LayerNormalization(name=f"ln_{i}")(x)
        x = tf.keras.layers.Dense(hidden_dim, activation='gelu', name=f"dense_{i}_1")(x)
        x = tf.keras.layers.Dense(embed_dim, name=f"dense_{i}_2")(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = x + residual
    
    # Final normalization
    x = tf.keras.layers.LayerNormalization(name="final_ln")(x)
    
    # Pool sequence (take last position or average)
    x = tf.keras.layers.GlobalAveragePooling1D(name="pool")(x)
    
    # Output projection to vocabulary
    outputs = tf.keras.layers.Dense(vocab_size, name="lm_head")(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="CerebrosLLM")
    return model

# Build model
model = build_cerebros_model(
    vocab_size=VOCAB_SIZE,
    max_seq_len=MAX_SEQ_LENGTH,
    embed_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS
)

model.summary()

## Prepare Training Data

Using sample text for demonstration. Replace with your own dataset.

In [ ]:
# Sample training text (replace with your dataset)
SAMPLE_TEXTS = [
    "In the beginning God created the heavens and the earth.",
    "The earth was without form and void, and darkness was over the face of the deep.",
    "And the Spirit of God was hovering over the face of the waters.",
    "And God said, Let there be light, and there was light.",
    "And God saw that the light was good. And God separated the light from the darkness.",
    "God called the light Day, and the darkness he called Night.",
    "And there was evening and there was morning, the first day.",
    "And God said, Let there be an expanse in the midst of the waters.",
    "And let it separate the waters from the waters.",
    "And God made the expanse and separated the waters.",
]

def prepare_training_data(texts, tokenizer, max_seq_len):
    """Prepare training data: input sequences and next-token labels."""
    input_ids_list = []
    labels_list = []
    
    for text in texts:
        tokens = tokenizer.encode(text, add_special_tokens=False)
        
        # Create sliding window samples
        for i in range(len(tokens) - 1):
            # Input: tokens up to position i, padded to max_seq_len
            input_seq = tokens[:i+1]
            if len(input_seq) < max_seq_len:
                input_seq = input_seq + [tokenizer.pad_token_id] * (max_seq_len - len(input_seq))
            else:
                input_seq = input_seq[-max_seq_len:]
            
            # Label: next token
            next_token = tokens[min(i+1, len(tokens)-1)]
            
            input_ids_list.append(input_seq)
            labels_list.append(next_token)
    
    return np.array(input_ids_list, dtype=np.int32), np.array(labels_list, dtype=np.int32)

# Prepare data
X_train, y_train = prepare_training_data(SAMPLE_TEXTS, tokenizer, MAX_SEQ_LENGTH)
print(f"Training samples: {len(X_train)}")
print(f"Input shape: {X_train.shape}")
print(f"Labels shape: {y_train.shape}")

## Train the Model

In [ ]:
# Compile model
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train
print(f"\n🏋️ Training for {EPOCHS} epochs...")
history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    verbose=1
)

print(f"\n✅ Training complete!")
print(f"Final loss: {history.history['loss'][-1]:.4f}")
print(f"Final accuracy: {history.history['accuracy'][-1]:.4f}")

## Save Keras Model

In [ ]:
# Save the Keras model
model.save(MODEL_PATH)
print(f"✅ Keras model saved to: {MODEL_PATH}")
print(f"   Size: {os.path.getsize(MODEL_PATH) / 1024:.1f} KB")

## Convert to ONNX

Export the trained model to ONNX format for cross-platform inference.

In [ ]:
import tf2onnx
import onnx

print("🔄 Converting to ONNX format...")

# Define input signature
input_signature = [
    tf.TensorSpec(shape=(None, MAX_SEQ_LENGTH), dtype=tf.int32, name="input_ids")
]

# Convert to ONNX
model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=14,
    output_path=ONNX_PATH
)

print(f"✅ ONNX model saved to: {ONNX_PATH}")
print(f"   Size: {os.path.getsize(ONNX_PATH) / 1024:.1f} KB")

# Verify the model
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)
print(f"✅ ONNX verification passed!")

# Print model info
print(f"\n📊 ONNX Model Info:")
print(f"   IR Version: {onnx_model.ir_version}")
print(f"   Opset Version: {onnx_model.opset_import[0].version}")

print(f"\n📥 Inputs:")
for inp in onnx_model.graph.input:
    shape = [d.dim_value if d.dim_value else 'batch' for d in inp.type.tensor_type.shape.dim]
    print(f"   {inp.name}: {shape}")

print(f"\n📤 Outputs:")
for out in onnx_model.graph.output:
    shape = [d.dim_value if d.dim_value else 'batch' for d in out.type.tensor_type.shape.dim]
    print(f"   {out.name}: {shape}")

## Test ONNX Inference

Verify the ONNX model works correctly.

In [ ]:
import onnxruntime as ort

print("🧪 Testing ONNX inference...")

# Create ONNX Runtime session
ort_session = ort.InferenceSession(ONNX_PATH)

# Get input/output names
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

print(f"   Input name: {input_name}")
print(f"   Output name: {output_name}")

# Test prompt
test_prompt = "In the beginning"
test_tokens = tokenizer.encode(test_prompt, add_special_tokens=False)

# Pad to MAX_SEQ_LENGTH
if len(test_tokens) < MAX_SEQ_LENGTH:
    test_tokens = test_tokens + [tokenizer.pad_token_id] * (MAX_SEQ_LENGTH - len(test_tokens))

test_input = np.array([test_tokens], dtype=np.int32)
print(f"\n📥 Test input: '{test_prompt}'")
print(f"   Token IDs: {test_tokens[:10]}...")

# Run ONNX inference
onnx_output = ort_session.run([output_name], {input_name: test_input})[0]
print(f"\n📤 ONNX output shape: {onnx_output.shape}")

# Get top predictions
probs = tf.nn.softmax(onnx_output[0]).numpy()
top_k = 5
top_indices = np.argsort(probs)[-top_k:][::-1]

print(f"\n🔝 Top {top_k} predicted next tokens:")
for idx in top_indices:
    token = tokenizer.decode([idx])
    print(f"   {repr(token)}: {probs[idx]:.4f}")

## Compare Keras vs ONNX Outputs

Verify the ONNX model produces equivalent results to Keras.

In [ ]:
# Run Keras inference
keras_output = model(tf.constant(test_input), training=False).numpy()

# Compare
max_diff = np.max(np.abs(onnx_output - keras_output))
mean_diff = np.mean(np.abs(onnx_output - keras_output))

print("📊 Keras vs ONNX Comparison:")
print(f"   Max absolute difference: {max_diff:.2e}")
print(f"   Mean absolute difference: {mean_diff:.2e}")

if max_diff < 1e-4:
    print("\n✅ ONNX outputs match Keras (within tolerance)")
else:
    print("\n⚠️  Some numerical differences (often acceptable for inference)")

## Text Generation (Optional)

Simple autoregressive generation using the ONNX model.

In [ ]:
def generate_text(prompt, max_new_tokens=20, temperature=0.7, top_k=50):
    """Generate text using the ONNX model."""
    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    
    for _ in range(max_new_tokens):
        # Prepare input
        input_tokens = tokens[-MAX_SEQ_LENGTH:]
        if len(input_tokens) < MAX_SEQ_LENGTH:
            input_tokens = input_tokens + [tokenizer.pad_token_id] * (MAX_SEQ_LENGTH - len(input_tokens))
        
        input_array = np.array([input_tokens], dtype=np.int32)
        
        # Run inference
        logits = ort_session.run([output_name], {input_name: input_array})[0][0]
        
        # Apply temperature
        logits = logits / temperature
        
        # Top-k sampling
        top_k_indices = np.argsort(logits)[-top_k:]
        top_k_logits = logits[top_k_indices]
        probs = tf.nn.softmax(top_k_logits).numpy()
        
        # Sample
        next_token_idx = np.random.choice(len(top_k_indices), p=probs)
        next_token = top_k_indices[next_token_idx]
        
        tokens.append(int(next_token))
        
        # Stop at EOS
        if next_token == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(tokens)

# Generate samples
print("🔮 Generation Samples:\n")

prompts = [
    "In the beginning",
    "And God said",
    "The earth was"
]

for prompt in prompts:
    generated = generate_text(prompt, max_new_tokens=15, temperature=0.8)
    print(f"Prompt: '{prompt}'")
    print(f"Output: '{generated}'\n")

## Summary

### Files Created
- `cerebros_model.keras` - Trained Keras model
- `cerebros_model.onnx` - ONNX export for cross-platform inference

### Usage in Other Languages

**Python (ONNX Runtime):**
```python
import onnxruntime as ort
session = ort.InferenceSession("cerebros_model.onnx")
output = session.run(["output"], {"input_ids": token_ids})
```

**Elixir (via Ortex):**
```elixir
{:ok, model} = Ortex.load("cerebros_model.onnx")
input = Nx.tensor([[token_ids...]], type: :s32)
{output} = Ortex.run(model, input)
```

**JavaScript (ONNX.js):**
```javascript
const session = await ort.InferenceSession.create('cerebros_model.onnx');
const output = await session.run({input_ids: tensorInput});
```

**C++ (ONNX Runtime C API):**
```cpp
Ort::Session session(env, "cerebros_model.onnx", session_options);
auto output = session.Run(...);
```

In [ ]:
# Final summary
print("="*60)
print("✅ COMPLETE: Cerebros LLM Training → ONNX Export")
print("="*60)
print(f"\n📁 Output Files:")
print(f"   {MODEL_PATH} ({os.path.getsize(MODEL_PATH)/1024:.1f} KB)")
print(f"   {ONNX_PATH} ({os.path.getsize(ONNX_PATH)/1024:.1f} KB)")
print(f"\n📊 Model Config:")
print(f"   Vocabulary: {VOCAB_SIZE:,}")
print(f"   Sequence Length: {MAX_SEQ_LENGTH}")
print(f"   Embedding Dim: {EMBEDDING_DIM}")
print(f"   Parameters: {model.count_params():,}")
print(f"\n🎯 ONNX Input: input_ids [batch, {MAX_SEQ_LENGTH}] int32")
print(f"🎯 ONNX Output: logits [batch, {VOCAB_SIZE}] float32")